In [ ]:
import os
import fitz
import re
import json

In [14]:
def normalize_text(text):
    return re.sub(r'[ \t]+', ' ', text.replace('\xa0', ' ')).strip()

def split_into_paragraphs(text):
    # Разделяем по основным пунктам (1. 2. 3. и т.д.)
    raw_sections = re.split(r'(?=\n?\d+\.)', text)
    sections = []

    for raw in raw_sections:
        raw = raw.strip()
        if not raw:
            continue
        match = re.match(r'^(\d+)\.\s*(.*)', raw, re.DOTALL)
        if not match:
            continue
        num, content = match.groups()
        # Выделяем подпункты: а), б), в)... или маркированные списки
        subs = re.findall(r'\n?([а-я]\))\s*(.*?)(?=(?:\n[а-я]\))|\n\d+\.)', content + '\n', re.DOTALL)
        subs_cleaned = [f"{s[0]} {normalize_text(s[1])}" for s in subs] if subs else []
        sections.append({
            "номер": f"{num}.",
            "текст": normalize_text(content.split('\n')[0]),
            "подпункты": subs_cleaned
        })

    return sections

def parse_pdf_to_json_19_10_2022_N_419(pdf_path):
    doc = fitz.open(pdf_path)
    full_text = ""
    for page in doc:
        full_text += page.get_text()

    normalized_text = normalize_text(full_text)
    sections = split_into_paragraphs(normalized_text)

    return {
        "документ": "Приказ Минтранса России от 19.10.2022 N 419 'О перечне специалистов авиационного персонала гражданской авиации Российской Федерации'",
        "главы": [
            {
                "глава": "I.",
                "заголовок": "ПЕРЕЧЕНЬ СПЕЦИАЛИСТОВ АВИАЦИОННОГО ПЕРСОНАЛА ГРАЖДАНСКОЙ АВИАЦИИ РОССИЙСКОЙ ФЕДЕРАЦИИ",
                "пункты": sections
            }
        ]
    }

# Сохраняем


In [16]:
parsed = parse_pdf_to_json_19_10_2022_N_419("C:\\Users\\My Computer\\Desktop\\Work\\Learn\\OPK_Project\\documents_pdf\\19_10_2022_N_419.pdf")
with open("C:\\Users\\My Computer\\Desktop\\Work\\Learn\\OPK_Project\\documents_json\\19_10_2022_N_419.json", "w", encoding="utf-8") as f:
    json.dump(parsed, f, ensure_ascii=False, indent=2)